# **はじめに**
　まずは，コンテスト主催関係者の皆様この度は，SIGNATE Student Cup 2021秋を開催頂きありがとうございました。また，参加者の皆様お疲れさまでした。

　PrivateLB 13位でしたが，記念に共有しようと思い，取り組んだことをまとめてみました。

# **方針**
## 特徴量
　trip.csv以外の特徴量を単純mergeし，加えたものはdateから分かる曜日や休日(祝日)か否かという特徴量のみになります。(今思うと，使用したツール的に，祝日かどうかというのは日本での話だったのですが、Local Publicともスコアがあがったので採用していました)カテゴリ変数については、Label Encodingで統一しました。

　その他主にweather.csvについてビニング処理を行った特徴量を加えましたが、PublicLBであまりスコアの向上が見られなかったので採用しませんでした(やり方が上手くないのかもしれない)。

## Validation方法
　チズチズさんやsylkさんがサンプルとして投稿してくださっていたように，訓練/検証/テストデータを1か月ずつスライドさせる方法で検証を行いました。

　独自に行ったこととしては，テスト用に含まれるデータに関して，予測日以前のデータを訓練データに加えて，学習を行うようにしました。例えば，2014/9月に予測対象日が10日間あれば，10個分の訓練データ+予測日以前のテストデータが用意され，10個のモデルで予測を行うということになります。

## モデル
　各ValidationでLightGBM, CatBoost, XGBRegressorの算術平均を予測値としました。

# **感想**
　テスト用の月のデータを訓練データに追加したサブミットは，Privateのスコアでほとんど3.53より小さい値が出ていたので、結構効果があったようです。直近のデータが訓練データに入っていた方が良さそうというのは直感的にもそうかなと感じています。

　trip.csvの取り扱いをどうするか悩みましたが、結局上手く使うことが出来ませんでした。各stationの使われやすさを示す様に，各date/hourごとにstart_stationやend_stationの件数を集計して特徴量に加えたところLocal CVは良くなりましたが，Publicとの乖離があったので採用しませんでした。どう使うのがいいのが知りたいです。

In [1]:
!pip3 install jpholiday
!pip install catboost
!pip install xgboost

     |████████████████████████████████| 76.3 MB 59 kB/s 


In [3]:
!cp /content/drive/MyDrive/SIGNATE/StudentCup2021autumn/station.csv .
!cp /content/drive/MyDrive/SIGNATE/StudentCup2021autumn/status.csv .
!cp /content/drive/MyDrive/SIGNATE/StudentCup2021autumn/weather.csv .
!cp /content/drive/MyDrive/SIGNATE/StudentCup2021autumn/trip.csv .
!cp /content/drive/MyDrive/SIGNATE/StudentCup2021autumn/sample_submit.csv .

# ライブラリのImport, Seedの固定

In [4]:
import random
import os
import torch

import numpy as np
import pandas as pd
import warnings
import pickle
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor
import xgboost as xgb

warnings.simplefilter('ignore')

def seed_torch(seed=42):
    # python の組み込み関数の seed を固定
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # numpy の seed を固定
    np.random.seed(seed)
    # torch の seed を固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # 決定論的アルゴリズムを使用する
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_torch(SEED)

# 教師データ読み込みと前処理

In [5]:
weather_df = pd.read_csv("weather.csv")
weather_df["date"] = pd.to_datetime(weather_df["date"])
# weather_df

In [6]:
station_df = pd.read_csv("station.csv")
# station_df

In [7]:
status_df = pd.read_csv("status.csv")
# status_df

In [8]:
#曜日を追加するための関数を定義
def get_weekday_jp(dt):
    w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
    return(w_list[dt.weekday()])

In [9]:
#year, month, dayを結合してdatetime型に変換
status_df["date"] = status_df["year"].astype(str) + status_df["month"].astype(str).str.zfill(2) + status_df["day"].astype(str).str.zfill(2)
status_df["date"] = pd.to_datetime(status_df["date"])

#dateから曜日情報を取得
status_df["weekday"] = status_df["date"].apply(get_weekday_jp)
# status_df

In [10]:
main_df = status_df.merge(station_df, left_on="station_id", right_on="station_id")
main_df = main_df.merge(weather_df, left_on="date", right_on="date")
# main_df

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

main_df["weekday"] = le.fit_transform(main_df["weekday"])
main_df["city"] = le.fit_transform(main_df["city"])
main_df["installation_date"] = le.fit_transform(main_df["installation_date"])
main_df["events"] = le.fit_transform(main_df["events"].fillna("NaN"))

print(main_df.columns)
print(main_df.shape)

Index(['id', 'year', 'month', 'day', 'hour', 'station_id', 'bikes_available',
       'predict', 'date', 'weekday', 'lat', 'long', 'dock_count', 'city',
       'installation_date', 'max_temperature', 'mean_temperature',
       'min_temperature', 'max_dew_point', 'mean_dew_point', 'min_dew_point',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure', 'mean_sea_level_pressure',
       'min_sea_level_pressure', 'max_visibility', 'mean_visibility',
       'min_visibility', 'max_wind_Speed', 'mean_wind_speed', 'precipitation',
       'cloud_cover', 'events', 'wind_dir_degrees'],
      dtype='object')
(1226400, 36)


In [12]:
import datetime
import jpholiday
from tqdm.notebook import tqdm

holiday = []

for i in tqdm(main_df["date"].dt.to_pydatetime()):
  holiday_name = jpholiday.is_holiday(i)
  if holiday_name:
    holiday.append(1)
  else:
    holiday.append(0)

  0%|          | 0/1226400 [00:00<?, ?it/s]

In [14]:
# holiday = pd.DataFrame(holiday, columns=["holiday"])
# 時間が掛かるのでDriveに保存
holiday = pd.read_csv("/content/drive/MyDrive/SIGNATE/StudentCup2021autumn/holiday.csv")
main_df = main_df.merge(holiday, left_index=True, right_index=True)

In [16]:
# main_df

# 学習用関数の定義

In [17]:
from dateutil.relativedelta import relativedelta

class TimeSeriesSplitGenerator:
    def __init__(self, n_split = 12, test_day_after = "2014-09-01", slide = False):
        self.test_day_after = pd.to_datetime(test_day_after)
        self.n_split = n_split
        self.test_month_period = 12
        self.month = relativedelta(months = 1)

        self.slide = slide

    def split(self, X):
        for m in range(self.test_month_period):
            test_month = self.test_day_after + relativedelta(months=m)
            test_index = (test_month <= X.date) & (X.date < test_month + self.month)
            valid_index = (test_month - self.month <= X.date) & (X.date < test_month)
            train_index = (X.date < test_month - self.month)
            if self.slide:
                train_index = train_index &(
                    test_month - self.month - relativedelta(months =12) <= X.date
                )
            yield train_index, valid_index, test_index

In [23]:
def lightgbm(X_train, Y_train, X_valid, Y_valid, X_test):
  bst_params = {
      "boosting_type": "gbdt",
      "metric": "rmse",
      "objective": "regression",
      "n_jobs": -1,
      "seed": SEED,
      'random_state': SEED,
      "learning_rate": 0.01,
      "bagging_fraction": 0.75,
      "bagging_freq": 10,
      "colsample_bytree": 0.75,
      "num_boost_round": 10000,
      "early_stopping_rounds": 10,
      "verbose_eval": 1000,
  }

  lgb_train = lgb.Dataset(X_train, Y_train)
  lgb_valid = lgb.Dataset(X_valid, Y_valid)
  
  model = lgb.train(bst_params, lgb_train,
                    valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                    verbose_eval=1000)

  # 検証データに対する予測値を求める
  va_pred = model.predict(X_valid, num_iteration=model.best_iteration)

  #テストデータに対する予測値を求める
  te_pred = np.array(model.predict(X_test, num_iteration=model.best_iteration))

  return va_pred, te_pred, model


def catboost(X_train, Y_train, X_valid, Y_valid, X_test):
  # objectの列番号を取得
  categorical_features_indices = np.where(X_train.dtypes==np.object)[0]
  lgb_train = Pool(X_train, Y_train, cat_features=categorical_features_indices)
  lgb_valid = Pool(X_valid, Y_valid, cat_features=categorical_features_indices)
  model = CatBoostRegressor(eval_metric='RMSE',
                            loss_function='RMSE',
                            num_boost_round=10000,
                            logging_level='Silent',
                            random_seed=SEED)
  model.fit(lgb_train, 
            eval_set=lgb_valid,
            early_stopping_rounds=10,
            verbose=True,
            use_best_model=True)

  # 検証データに対する予測値を求める
  va_pred = model.predict(X_valid)

  mse = mean_squared_error(Y_valid, va_pred)
  rmse = np.sqrt(mse) # RSME = √MSEの算出
  eval_metric = rmse

  print(f"eval's rmse: {eval_metric}")

  #テストデータに対する予測値を求める
  te_pred = np.array(model.predict(X_test))

  return va_pred, te_pred, model

def xgboost(X_train, Y_train, X_valid, Y_valid, X_test):

  xgb_params = {
      'objective': 'reg:linear',
      'eval_metric': 'rmse',
      # "verbosity": 0,
      "seed": SEED,
      "eta": 0.01,
      "num_boost_round": 10000,
      # "early_stopping_rounds": 10,
      # "verbose_eval": 100,
  }

  lgb_train = xgb.DMatrix(X_train, label=Y_train)
  lgb_valid = xgb.DMatrix(X_valid, label=Y_valid)
  lgb_test = xgb.DMatrix(X_test)
  evals = [(lgb_train, 'train'), (lgb_valid, 'eval')]
  evals_result = {}

  model = xgb.train(xgb_params,
                    lgb_train,
                    evals=evals,
                    evals_result=evals_result,
                    num_boost_round=10000,
                    early_stopping_rounds=10,
                    verbose_eval=1000,
                  )
  
  # 検証データに対する予測値を求める
  va_pred = model.predict(lgb_valid)

  #テストデータに対する予測値を求める
  te_pred = list(model.predict(lgb_test))

  return va_pred, te_pred, model

# 学習と予測

最初に学習済みモデル保存用にフォルダを作成しておく

In [ ]:
scores = []
ids = []
submission = []

OUTPUT = "/content/drive/MyDrive/SIGNATE/StudentCup2021autumn/20211106"

for i, data in enumerate(TimeSeriesSplitGenerator(slide = True).split(main_df)):
    train_index, valid_index, test_index = data

    print("-------------------------------------------")
    print("train:", main_df[train_index].date.min(), main_df[train_index].date.max())
    print("valid:", main_df[valid_index].date.min(), main_df[valid_index].date.max())
    print("test: ", main_df[test_index].date.min(), main_df[test_index].date.max())
    print(f'Fold : {i}')

    train = main_df[train_index].dropna(subset=["bikes_available"])
    valid = main_df[valid_index].dropna(subset=["bikes_available"])

    test = main_df[test_index]
    test0 = test[test["predict"] == 0]
    test1 = test[test["predict"] == 1]
    
    rmses = []
    # 予測対象日について、一日ずつモデルを作成していく
    for start_date in test1["date"].unique():
      train_from_test = test0[test0["date"] < start_date].dropna(subset=["bikes_available"])
      train_add = pd.concat([train, train_from_test], axis=0)

      X_train, Y_train = train_add.drop(columns=["id", "predict", "bikes_available", "date"]), train_add["bikes_available"]
      X_valid, Y_valid = valid.drop(columns=["id", "predict", "bikes_available", "date"]), valid["bikes_available"]
      ids += list(test1[test1["date"] == start_date]["id"])
      X_test, _ = test1[test1["date"] == start_date].drop(columns=["id", "predict", "bikes_available", "date"]), test["bikes_available"]
      
      va_pred1, te_pred1, model = lightgbm(X_train, Y_train, X_valid, Y_valid, X_test)
      pickle.dump(model, open(f"{OUTPUT}/lgbm/{start_date}.pkl", 'wb'))
      va_pred2, te_pred2, model = catboost(X_train, Y_train, X_valid, Y_valid, X_test)
      pickle.dump(model, open(f"{OUTPUT}/cat/{start_date}.pkl", 'wb'))
      va_pred3, te_pred3, model = xgboost(X_train, Y_train, X_valid, Y_valid, X_test)
      pickle.dump(model, open(f"{OUTPUT}/xgb/{start_date}.pkl", 'wb'))
      
      va_pred = (va_pred1 + va_pred2 + va_pred3) / 3
      te_pred = (te_pred1 + te_pred2 + te_pred3) / 3

      # RSME = √MSEの算出
      mse = mean_squared_error(Y_valid, va_pred)
      rmse = np.sqrt(mse)

      rmses.append(rmse)

      #テストデータに対する予測値を求める
      submission += list(te_pred)
      print('')
      print(f"Fold: {i} {start_date} RMSE:{rmse}")
      print("")

    # フォールド毎の検証時のスコアを格納
    scores.append(np.mean(rmses))
    
    print('')
    print('################################')
    print(f"Fold: {i} RMSE:{np.mean(rmses)}")
    print("")
  
print(f"CV: {np.mean(scores)}")

# CV: 3.4555098

In [ ]:
sub = pd.Series(submission)
sub.index = ids
sub.to_csv(f"{OUTPUT}/submmission_ensenble.csv", header=False)

改めて関係者の皆様・参加者の皆様，ありがとうございました。とても楽しく取り組むことが出来ました。